<p style = "font-size : 30px; color : white ; text-align : left; background-color : #c0c0ff; border-radius: 5px 5px; padding : 10px"><img src="https://upload.wikimedia.org/wikipedia/commons/d/dd/Booking.com_Logo.svg" style="width:368px;height:81px;"></p>

<style>
a {
    color: #a63bff !important;
    font-size: 25px !important;
}
</style>
* <a href="#0">0. Introduction</a>
* <a href="#1">1. Start</a>
* <a href="#2">2. Исследование данных</a>
  * <a href="#2.1">2.1 Объединение данных</a>
* <a href="#3">3. Проектирование данных</a>
  * <a href="#3.1">3.1 Анализ дат </a>
  * <a href="#3.2">3.2 Анализ отелей </a>
  * <a href="#3.3">3.3 Координаты расположения отелей</a>
    * <a href="#3.3.1">3.3.1 Расстояние до центра</a>
  * <a href="#3.4">3.4 Информация о рецензенте</a>
  * <a href="#3.5">3.5 Обработка отзывов</a>
* <a href="#4">4. Очистка данных</a>
* <a href="#5">5. Model</a>
  * <a href="#5.1">5.1 Получение средней абсолютной ошибки</a>
* <a href="#6">6. Conclusion</a>
* <a href="#7">7. Submission </a>  

<a id = '0'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>0. Introduction</strong>
</p>

<p style="color: #ac67ff; font-size: 35px;">
    <strong>Задача проекта</strong>
</p>


<span style="color:#b29aff; font-family: 'Microsoft Sans Serif', Arial, sans-serif; font-size: 20px">
  
  В этом проекте мы будем работать с датасетом, в котором содержатся сведения о более чем 515 000 отзывов на отели Европы и Великобритании. Модель, которую мы будете обучать, должна предсказывать рейтинг отеля по данным сайта <b style="color:#a63bff"><strong>Booking.com</strong></b> на основе имеющихся в датасете данных.
    
<b style="color:#a63bff; font-size: 30px"><strong>Описание данных</strong></b>
    
    
<table >
   <tr>
    <th>Признак</th>
    <th>Описание</th>
  </tr>
  
   <tr>
    <td>hotel_address</td>
    <td>street, post code, city, country </td>
  </tr>
    
   <tr>
    <td>additional_number_of_scoring</td>
    <td>the number of hotel scores without review </td>
  </tr> 
    
  <tr>
    <td>review_date</td>
    <td>the day of review </td>
  </tr>
    
  <tr>
    <td>average_score</td>
    <td>the average rating of the hotel </td>
  </tr>
    
   <tr>
    <td>hotel_name</td>
    <td>the full name of hotel </td>
  </tr>
    
   <tr>
    <td>reviewer_nationality</td>
    <td>country from which the reviewer came </td>
  </tr>
    
   <tr>
    <td>negative_review </td>
    <td>text of negative review </td>
  </tr> 
   
   <tr>
    <td>review_total_negative_word_counts </td>
    <td>the total number words of negative review </td>
  </tr>  
    
   <tr>
    <td>total_number_of_reviews </td>
    <td>the total number of reviews that the hotel has </td>
  </tr>  
    
   <tr>
    <td>positive_review</td>
    <td>text of positive review </td>
  </tr> 
    
   <tr>
    <td>review_total_positive_word_counts</td>
    <td>the total number words of positive review </td>
  </tr> 
    
  <tr>
    <td>total_number_of_reviews_reviewer_has_given</td>
    <td>the total number of reviews reviewer has given </td>
  </tr> 
    
   <tr>
    <td>reviewer_score</td>
    <td>the number of reviewer score </td>
  </tr> 
    
   <tr>
    <td>tags</td>
    <td>tags that describe purpose of trip,type of room, count of nights of reviewer </td>
  </tr> 
    
   <tr>
    <td>days_since_review</td>
    <td>a difference in the number of days between review date and scrape date</td>
  </tr> 
    
   <tr>
    <td>lat</td>
    <td>latitude coordinate of hotel location </td>
  </tr> 
    
   <tr>
    <td>lng</td>
    <td>longitude coordinate of hotel location </td>
  </tr> 
    
</table>


</span>

<a id = '1'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>1. Start</strong>
</p>

In [ ]:
%%capture

##############################################

# https://www.kaggle.com/code/fobosal/project-3-booking

# установка дополнительных библиотек:
# !pip install Afinn
# !pip install nltk[twitter]
# !pip install wordcloud
# !pip install spacy 

#python -m spacy download en_core_web_lg
#python -m spacy download en_core_web_sm


import pandas as pd
import numpy as np
import re
import geopy.distance
from scipy import stats
from category_encoders import  OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px
%matplotlib inline

from wordcloud import WordCloud

# инструменты для лингвистического анализа:
import string
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob, Blobber
from textblob.sentiments import NaiveBayesAnalyzer
from afinn import Afinn
nltk.download('vader_lexicon')

import warnings
pd.options.mode.chained_assignment = None # SettingWithCopyWarning ignore
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning ignore

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
# Для воспроизводимости зафиксируем RANDOM_SEED
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
#!pip freeze > requirements.txt

In [ ]:
# данные из соревнования
#DATA_DIR = '/kaggle/input/sf-booking/'
#df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') 
#df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') 
#sample_submission = pd.read_csv(DATA_DIR+'/submission.csv')

#DATA_DIR = 'data/sf-booking'
#df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') 
#df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') 
#sample_submission = pd.read_csv(DATA_DIR+'/submission.csv')

DATA_DIR = 'P:/_Data Scientist DSPR-107/EDA-7. PROJECT-3. Feature Engineering. Соревнование на Kaggle/data/sf-booking'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') 
df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') 
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv')


<a id = '2'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>2. Исследование данных</strong>
</p>

In [ ]:
new_df = df_train[[df_train.columns[4], df_train.columns[0]]]

In [ ]:
df1 = df_train[['hotel_name', 'hotel_address']]

In [ ]:
# Создание нового DataFrame с уникальными значениями
unique_df = df1.drop_duplicates()

# Группировка данных и сбор адресов в список для каждого отеля
grouped_addresses = unique_df.groupby('hotel_name')['hotel_address'].apply(list)

# Поиск отеля с двумя разными адресами
for hotel, addresses in grouped_addresses.iteritems():
    if len(set(addresses)) > 1:
        print("Отель:\n", hotel)
        print("Адреса:\n", addresses)
        print("-------------------------------")

In [ ]:
grouped1_addresses = unique_df.groupby('hotel_address')['hotel_name'].apply(list)

# Поиск отеля с двумя разными адресами
for address, hotel in grouped1_addresses.iteritems():
    if len(set(hotel)) > 1:
        print("hotel:\n", hotel)
        print("address:\n", address)
        print("-------------------------------")

In [ ]:
df_train.info()

In [ ]:
df_train.nunique()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# Подсчитываем количество дубликатов
количество_дубликатов = df_train.duplicated().sum()

print("Количество дубликатов:", количество_дубликатов)

In [ ]:
# Фильтруем и выводим все дубликаты
дубликаты = df_train[df_train.duplicated(keep=False)]
display(дубликаты)

In [ ]:
дубликаты.info()

In [ ]:
# Подсчитываем количество дубликатов
количество_дубликатов = df_test.duplicated().sum()

print("Количество дубликатов:", количество_дубликатов)

<a id = '2.1'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>2.1 Объединение данных</strong>
</p>

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

# объединяем данные в один dataset
df = pd.concat([df_train, df_test], ignore_index=True)
#df['sample'] = df['sample'].astype('category')

In [ ]:
df.info()

In [ ]:
data = df.copy()
data.info()

In [ ]:
#data1 = df.copy()
#data1.info()

In [ ]:
#data = data1.drop_duplicates()

In [ ]:
data.columns

In [ ]:
#в каких столбцах числовые значения:
digit_columns = [s for s in data.columns if (data[s].dtypes == 'int64') | (data[s].dtypes == 'float64')]
digit_columns

In [ ]:
#в каких столбцах строковые значения:
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
object_columns

In [ ]:
data.info()

<a id = '3'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>3. Проектирование данных</strong>
</p>

<a id = '3.1'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>3.1 Анализ дат</strong>
</p>

In [ ]:
# Создадим новые признаки из даты: год, месяц
data['review_date'] = pd.to_datetime(data['review_date'])
data['month']=pd.to_datetime(data['review_date']).dt.month
data['year']=pd.to_datetime(data['review_date']).dt.year

In [ ]:
# Сгрупируем по отелям и посмотрим у каких отелей отсутствуют геоданные
hotels_with_coor = data[data.lng.isna() & data.lat.isna()].groupby(by = ['hotel_address','hotel_name']).\
  count().iloc[:,0].to_frame().reset_index()
hotels_with_coor.head(2)

<a id = '3.2'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>3.2 Анализ отелей</strong>
</p>

In [ ]:
data['hotel_address'].unique()

Комментарии / выводы:

Из анализа созданных новых признаков мы видим, что отели распологаются в 6 странах и в 6 крупных городах - туристических центрах. Количественные данные совпадают. Для построения модели ML предлагается произвести кодировку по методу однократного кодирования OneHot.

In [ ]:
# исследуем данные о названии на наличие пропусков и дубликатов:
pivot_n = data[['hotel_address', 'hotel_name']].groupby(
    by='hotel_name'
).nunique()

name_rep = pivot_n[pivot_n['hotel_address'] != 1].index.to_list()
mask = data['hotel_name'].isin(name_rep)
addresses_rep = data[mask][['hotel_name', 'hotel_address']].value_counts()

print('Дублирование названия отеля:')
display(addresses_rep)

# чтобы различать эти отели при анализе - добавим в название город:
data['hotel_name'] = data.apply(
    lambda row: (row['hotel_name'] + ' ' + row['hotel_address'].split()[-2])
    if (row['hotel_name'] in name_rep)
    else row['hotel_name'], axis=1)

print('\nПроверим полученные изменения:')
mask = data['hotel_name'].str.contains(name_rep[0])
display(data[mask]['hotel_name'].value_counts())

In [ ]:
#выбираем страну из адреса отеля
data['hotel_country'] = data['hotel_address'].apply(
    lambda x: 'United Kingdom' if x.split()[-1] == 'Kingdom' else x.split()[-1])

In [ ]:
# Для распечатки в цвете
from termcolor import colored

data['hotel_city'] = data['hotel_address'].apply\
(lambda x: x.split()[-5] if x.split()[-1] == 'Kingdom' else x.split()[-2])

print(colored('\n Какие города получились в датасете:', 'green'))
data['hotel_city'].value_counts()


In [ ]:
#закодируем название стран
import category_encoders as ce
encoder = ce.OneHotEncoder()
data['make_hotel_name'] = data['hotel_country'].astype('category')
data_bin = encoder.fit_transform(data['hotel_country'])
data_bin.rename(columns={'hotel_country' : 'hotel_country_c'}, inplace=True)
data = pd.concat([data, data_bin], axis=1)

In [ ]:
data.nunique(dropna=False)

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [ ]:
data.sample(3)

<a id = '3.3'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>3.3 Координаты расположения отелей </strong>
</p>

In [ ]:
#поиск пропусков в данных

null_data = data.isnull().sum()

print(f'\nСтолбцы с пропусками данных:\n{null_data[null_data > 0]}')

In [ ]:
# Нехитрым способом проверим, что пропуски по геоданным у одних и тех же записей.
# Как видим, мы выбрали данные где одновременно присутствует пропуски по обоим признакам,
# отняли от общего кол-ва записей и кол-во оставшихся записей точно равно данным из data.info()
data[(data.lng.isna()) & (data.lat.isna())].shape[0] - data.shape[0]

In [ ]:
# формируем словарь с данными по широте
dict_lat = {'Austria Trend Hotel Schloss Wilhelminenberg Wien': '48.21974808057666', 
                'Holiday Inn Paris Montmartre': '48.889064368690235',
                'Fleming s Selection Hotel Wien City': '48.20949471863499',
                'Hotel Atlanta': '48.2209723965124',
                'Maison Albar Hotel Paris Op ra Diamond': '48.87543722920124',
                'Derag Livinghotel Kaiser Franz Joseph Vienna': '48.24608017805536',
                'Hotel City Central': '48.214201786014854',
                'NH Collection Barcelona Podium': '41.39176529029773',
                'City Hotel Deutschmeister': '48.22103057609981',
                'Hotel Daniel Vienna': '48.18897212994251',
                'Roomz Vienna': '48.22382961585223',
                'Hotel Pension Baron am Schottentor': '48.21692824899107',
                'Mercure Paris Gare Montparnasse': '48.84010021974514',
                'Hotel Park Villa': '48.23759024291264',
                'Hotel Advance': '41.38327316014033',
                'Renaissance Barcelona Hotel': '41.394162363243964',
                'Cordial Theaterhotel Wien': '48.20969901650749'}

In [ ]:
# формируем словарь с данными по долготе
dict_lng = {'Austria Trend Hotel Schloss Wilhelminenberg Wien': '16.285624283904383', 
                'Holiday Inn Paris Montmartre': '2.333206641604505',
                'Fleming s Selection Hotel Wien City': '16.35350997041095',
                'Hotel Atlanta': '16.356650516337886',
                'Maison Albar Hotel Paris Op ra Diamond': '2.3234027262612256',
                'Derag Livinghotel Kaiser Franz Joseph Vienna': '16.34202545691955',
                'Hotel City Central': '16.379923233140797',
                'NH Collection Barcelona Podium': '2.1783660658934747',
                'City Hotel Deutschmeister': '16.366622226233126',
                'Hotel Daniel Vienna': '16.3838329415743',
                'Roomz Vienna': '16.393969343883455',
                'Hotel Pension Baron am Schottentor': '16.360055901096636',
                'Mercure Paris Gare Montparnasse': '2.3231778953046462',
                'Hotel Park Villa': '16.34703938556462',
                'Hotel Advance': '2.162964217594162',
                'Renaissance Barcelona Hotel': '2.1748749075722924',
                'Cordial Theaterhotel Wien': '16.351495483903957'}

In [ ]:
# формируем код и производим заполнение данных в dataframe
data['lat'] = data.apply(lambda x: dict_lat.get(x['hotel_name']) if pd.isnull(x['lat']) else x['lat'], axis=1).astype('float64') # вводим данные по широте и устанавливаем тип float64
data['lng'] = data.apply(lambda x: dict_lng.get(x['hotel_name']) if pd.isnull(x['lng']) else x['lng'], axis=1).astype('float64') # вводим данные по долготе и устанавливаем тип float64

In [ ]:
# формируем и выводим повторно информацию о наличие пропусков после обработки
display(data.isnull().sum())

<a id = '3.3.1'></a>
<p style="color: #b29aff; font-size: 40px;">
    <strong>3.3.1 Расстояние до центра</strong>
</p>

Можно рассчитать расстояние от отеля до центра города, возможно это влияет на оценку отеля.

In [ ]:
%%time

import geopy.distance

# для формирования нового признака используются координаты центра каждого города 

centers = pd.DataFrame(
    data = [
        ['United Kingdom', 'London', 51.5085300, -0.1257400],
        ['France', 'Paris', 48.8534100, 2.3488000],
        ['Spain', 'Barcelona', 41.3887900, 2.1589900],
        ['Italy', 'Milan', 45.4642700, 9.1895100],
        ['Austria', 'Vienna', 48.2084900, 16.3720800],
        ['Netherlands', 'Amsterdam', 52.3740300, 4.8896900],
    ],
    columns= ['country', 'city', 'center_lat', 'center_lng']
).set_index('country')

#функция вычисления до центра города по координатам
def get_dist_to_center(country, lng, lat):
   
    center_lat = centers['center_lat'][country]
    center_lng = centers['center_lng'][country]

    return geopy.distance.geodesic([center_lng, center_lat], [lng, lat]).km

hotels_coord = data[['hotel_name', 'hotel_country', 'lng', 'lat']].drop_duplicates()
hotels_coord['hotel_distance_to_center'] = data.apply(
    lambda row: get_dist_to_center(row['hotel_country'], row['lng'], row['lat']), axis=1)

hotels_coord.drop(['hotel_country', 'lng', 'lat'], axis=1, inplace=True)

data = data.merge(hotels_coord, how='left', on='hotel_name', sort=False)

In [ ]:
#посмотрим новый признак
data[['hotel_country', 'hotel_distance_to_center', 'reviewer_score']].sort_values(by='hotel_country').iloc[:10]

In [ ]:
data.info()

In [ ]:
# Сгрупируем по отелям и посмотрим у каких отелей отсутствуют геоданные
hotels_with_coor = data[data.lng.isna() & data.lat.isna()].groupby(by = ['hotel_address','hotel_name']).\
  count().iloc[:,0].to_frame().reset_index()
hotels_with_coor.head(2)

<a id = '3.4'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>3.4 Информация о рецензенте</strong>
</p>

In [ ]:
data['reviewer_nationality'].describe()

In [ ]:
#сгруппириуем данные для оценки параметра национальность
data1 = data[data['sample'] == 1]
data1 = data1.groupby('reviewer_nationality')['reviewer_score'].agg('mean').reset_index().sort_values(by='reviewer_score').iloc[:20]
data1.head()

In [ ]:
from colorama import Fore, Back, Style
# Null записей нету, но проверим на всякий случай, нет ли пустых записей
cnt_no_nationality = data[data.reviewer_nationality.str.strip() == ''].shape[0]
print(Fore.LIGHTCYAN_EX + "\nКоличество записей без указанной национальности reviewer-а:",cnt_no_nationality)

Количество записей без указанной национальности reviewer-а: 523

In [ ]:
# заполним модой
mode_country_reviewer = data.reviewer_nationality.mode().item()
data.loc[data[data.reviewer_nationality.str.strip() == ''].index,'reviewer_nationality'] = mode_country_reviewer

In [ ]:
# Наш топ 122. Почему 122, просто решил что примерно 50 отзывов, это минимальное, что 
# может нести в себе более менее усредненную информацию
countries_top = data['reviewer_nationality'].value_counts().nlargest(122)
countries_top.index = countries_top.index.map(str.strip)
countries_top

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 20))
sns.histplot(
    data = data1,
    x = 'reviewer_score',
    bins = 30,
    kde = True,
    ax = axes[0]
);
axes[0].set_title('Диаграмма оценок и национальностей туристов ', fontsize = 14)
sns.histplot(
    data = data1,
    x = 'reviewer_score',
    y = 'reviewer_nationality',
    bins = 30,
    color = 'red',
    ax = axes[1]
);

In [ ]:
#закодируем название национальностей
import category_encoders as ce
ord_encoder = ce.OrdinalEncoder()
data['make_nationality'] = data['reviewer_nationality'].astype('category')
data_bin = ord_encoder.fit_transform(data['reviewer_nationality'])
data_bin.rename(columns={'reviewer_nationality' : 'nationality'}, inplace=True)
data = pd.concat([data, data_bin], axis=1)

In [ ]:
data[['reviewer_nationality', 'nationality']].value_counts()

In [ ]:
data.info()

<a id = '3.5'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>3.5 Обработка отзывов</strong>
</p>

In [ ]:
%%time

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Первым делов подправим явные выражения 
data['negative_review'] = data['negative_review'].apply(lambda x: x.replace('No Negative', 'Positive'))
data['negative_review'] = data['negative_review'].apply(lambda x: x.replace('N A', 'Positive'))
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.replace('All good', 'Positive'))
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.replace('No complaints', 'Positive'))
data['negative_review'] = data['negative_review'].apply(
    lambda x: x.replace('Nothing to dislike', 'Positive'))
data['positive_review'] = data['positive_review'].apply(
    lambda x: x.replace('No Positive', 'Negative'))

# создаем два экземпляра анализатора, чтобы разделить лексикон 
# для разной системы оценки положительных и отрицательных отзывов:
sia_neg = SentimentIntensityAnalyzer()
sia_pos = SentimentIntensityAnalyzer()

data['neg_scores'] = data['negative_review'].apply(lambda x: sia_neg.polarity_scores(x))
data['pos_scores'] = data['positive_review'].apply(lambda x: sia_pos.polarity_scores(x))

# формируем новые признаки:
data['n_review_sentiments_neg'] = data['neg_scores'].apply(lambda x: x['neg'])
data['n_review_sentiments_neu'] = data['neg_scores'].apply(lambda x: x['neu'])
data['n_review_sentiments_pos'] = data['neg_scores'].apply(lambda x: x['pos'])
data['n_review_sentiments_compound'] = data['neg_scores'].apply(lambda x: x['compound'])

data['p_review_sentiments_neg'] = data['pos_scores'].apply(lambda x: x['neg'])
data['p_review_sentiments_neu'] = data['pos_scores'].apply(lambda x: x['neu'])
data['p_review_sentiments_pos'] = data['pos_scores'].apply(lambda x: x['pos'])
data['p_review_sentiments_compound'] = data['pos_scores'].apply(lambda x: x['compound'])


In [ ]:
data.info()

<a id = '4'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>4. Очистка данных</strong>
</p>

In [ ]:
data.info()

In [ ]:
data.drop(['lat','lng'], axis = 1, inplace=True)

In [ ]:
#data.drop('year', axis = 1, inplace=True)

In [ ]:
#del data['make_hotel_name']

In [ ]:
#del data['make_nationality']

In [ ]:
# посмотрим признаки, которые типов категория, дата, и строки
def getNonNumberTypeFeatures(df):
    """Функция возвращает в списке все признаки из датасета, что подходят под описанные правила

    Args:
        df (pandas.Dataframe()): Датасет

    Returns:
        list: Список признаков
    """
    object_columns = [s for s in data.columns
                    if ((data[s].dtypes == 'object') | \
                        (data[s].dtype.name == 'category') | \
                        (data[s].dtypes == 'datetime64[ns]') |
                        (data[s].dtype.name == 'period[M]'))]
    return object_columns

types_list = getNonNumberTypeFeatures(data)
#print(Fore.LIGHTCYAN_EX + "\nКоличество признаков, которые надо удалить,\
# либо привести к машиноподходящему типу:",Fore.LIGHTGREEN_EX, types_list.__len__())
display(types_list)

In [ ]:
data.drop(types_list, axis = 1, inplace=True)

In [ ]:
data.sample(5)

In [ ]:
data.info()

<a id = '5'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>5. Model</strong>
</p>

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
%%time
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

<a id = '5.1'></a>
<p style="color: #b29aff; font-size: 45px;">
    <strong>5.1 Получение средней абсолютной ошибки</strong>
</p>

In [ ]:
# сравниваем предсказанные значения (y_pred) с реальными (y_test), 
# метрика Mean Absolute Percentage Error (MAPE) показывает среднюю абсолютную 
# процентную ошибку предсказанных значений от фактических:

def mean_absolute_percentage_error(y_tr, y_pr):
    """Получение средней абсолютной ошибки"""
    y_tr, y_pr = np.array(y_tr), np.array(y_pr)
    return np.mean(np.abs((y_tr - y_pr) / y_tr)) * 100

print('MAPE:', round(mean_absolute_percentage_error(y_test, y_pred), 2))

<a id = '6'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>6. Conclusion</strong>
</p>

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

<a id = '7'></a>
<p style = "background-color : #ac67ff; border-radius: 5px 5px; padding : 10px">
<p style="color: #b29aff; font-size: 50px;">
    <strong>7. Submission</strong>
</p>

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)